In [46]:
# vishesh : This is device env var dont change it in below cells 
import torch
device=torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device

device(type='mps')

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from datetime import datetime
import os
import pickle
from scipy.sparse import csr_matrix
import operator

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
class Scoreformer(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_feedforward, input_dim, num_weights=10, use_weights=True, dropout=0.1):
        super(Scoreformer, self).__init__()
        self.num_weights = num_weights
        self.use_weights = use_weights
        self.input_dim = input_dim
        self.d_model = d_model
        
        # Input projection layers
        self.input_linear = Linear(input_dim, d_model)
        self.dng_projection = Linear(input_dim, d_model)
        
        self.encoder_layer = TransformerEncoderLayer(
            d_model=d_model, 
            nhead=num_heads, 
            dim_feedforward=d_feedforward, 
            dropout=dropout, 
            batch_first=True
        )
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        
        # Final output layers
        self.pre_output = Linear(d_model, d_model)
        self.output_linear = Linear(d_model, 1)
        
        self.dropout = Dropout(dropout)
        self.layer_norm = LayerNorm(d_model)
        
        if self.use_weights:
            self.weight_linears = ModuleList([Linear(input_dim, d_model) for _ in range(num_weights)])

    def compute_neighborhood_similarity(self, adjacency_matrix, x):
        try:
            binary_adj = (adjacency_matrix > 0).float()
            intersection = binary_adj @ binary_adj.T
            row_sums = binary_adj.sum(dim=1, keepdim=True)
            col_sums = binary_adj.sum(dim=0, keepdim=True)
            union = row_sums + col_sums.T - intersection
            similarity = intersection / (union + 1e-8)
            return similarity @ x
        except RuntimeError:
            return torch.zeros_like(x)

    def project_graph_metrics(self, graph_metrics, target_dim):
        if graph_metrics.size(1) < target_dim:
            repeats = (target_dim + graph_metrics.size(1) - 1) // graph_metrics.size(1)
            graph_metrics = graph_metrics.repeat(1, repeats)[:, :target_dim]
        elif graph_metrics.size(1) > target_dim:
            graph_metrics = graph_metrics[:, :target_dim]
        return graph_metrics

    def forward(self, x, adjacency_matrix, graph_metrics, weights=None):
        adjacency_matrix = adjacency_matrix.float()
        graph_metrics = graph_metrics.float()
        batch_size, input_dim = x.shape
        
        if adjacency_matrix.size(0) != batch_size or adjacency_matrix.size(1) != batch_size:
            adjacency_matrix = torch.eye(batch_size, device=x.device)

        try:
            # Direct connections
            direct_scores = adjacency_matrix @ x
            
            # Neighborhood similarity
            neighborhood_similarity = self.compute_neighborhood_similarity(adjacency_matrix, x)
            
            # Graph structure scores
            if graph_metrics.dim() == 2:
                graph_metrics_projected = self.project_graph_metrics(graph_metrics, input_dim)
                graph_structure_scores = graph_metrics_projected * x
            else:
                graph_structure_scores = torch.zeros_like(x)

            # Combine DNG scores and project to d_model dimension
            dng_scores = direct_scores + neighborhood_similarity + graph_structure_scores
            dng_scores = self.dng_projection(dng_scores)  # Project to d_model dimension
            
            # Process input through transformer
            if self.use_weights and weights is not None:
                weighted_x = torch.zeros_like(x)
                for i, weight in enumerate(weights.T):
                    weighted_x += self.weight_linears[i](x) * weight.unsqueeze(1)
                transformer_input = weighted_x
            else:
                transformer_input = self.input_linear(x)  # Project to d_model dimension

            # Apply transformer
            transformer_input = self.layer_norm(transformer_input)
            transformer_output = self.transformer_encoder(transformer_input.unsqueeze(1)).squeeze(1)
            
            # Combine transformer output with DNG scores
            combined = transformer_output + dng_scores
            combined = self.dropout(combined)
            
            # Final output processing
            output = self.pre_output(combined)
            output = F.relu(output)
            output = self.output_linear(output)
            
            return output.squeeze(-1)  # Return [batch_size] tensor

        except RuntimeError as e:
            print(f"RuntimeError during forward pass: {e}")
            print(f"x shape: {x.shape}, adjacency_matrix shape: {adjacency_matrix.shape}, graph_metrics shape: {graph_metrics.shape}")
            raise

In [17]:
class MultiBehaviorDataset:
    def __init__(self, data_path='/Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/beibei01/', max_users=10000):
        self.data_path = data_path
        self.behaviors = ['pv', 'cart', 'buy']
        self.trn_file = data_path + 'trn_'
        self.tst_file = data_path + 'tst_'
        self.max_users = max_users  # Limit number of users
        
        self.load_training_data()
        self.load_test_data()
    
    def load_training_data(self):
        print("Loading training data from:", self.data_path)
        self.trn_mats = []
        for beh in self.behaviors:
            path = self.trn_file + beh
            print(f"Loading behavior: {beh} from {path}")
            if not os.path.exists(path):
                raise FileNotFoundError(f"File not found: {path}")
            with open(path, 'rb') as fs:
                mat = pickle.load(fs)
                if not isinstance(mat, csr_matrix):
                    mat = csr_matrix(mat)
                mat = (mat != 0).astype(np.float32)
                self.trn_mats.append(mat)
        
        self.trn_label = 1 * (self.trn_mats[-1] != 0)
        self.n_users, self.n_items = self.trn_mats[0].shape
        self.n_behaviors = len(self.behaviors)
        print(f"Dataset dimensions: {self.n_users} users, {self.n_items} items")

    def load_test_data(self):
        """Load test data"""
        test_path = self.tst_file + 'int'
        print(f"Loading test data from: {test_path}")
        if not os.path.exists(test_path):
            raise FileNotFoundError(f"File not found: {test_path}")
        with open(test_path, 'rb') as fs:
            self.tst_int = np.array(pickle.load(fs))
        
        self.tst_users = np.reshape(np.argwhere(self.tst_int != None), [-1])
        # Limit test users for faster processing
        self.tst_users = self.tst_users[:min(len(self.tst_users), self.max_users)]
        print(f"Using {len(self.tst_users)} test users")

    def create_adjacency_matrix(self, users):
        """Optimized adjacency matrix creation with MPS support"""
        # Move users to CPU for numpy/scipy operations
        users_cpu = users.cpu()
        batch_size = len(users_cpu)
        adj_matrix = torch.zeros(batch_size, batch_size)
        
        # Pre-compute user item sets
        user_item_sets = []
        for user in users_cpu:
            user_items = set()
            for mat in self.trn_mats:
                user_items.update(mat[user.item()].indices)
            user_item_sets.append(user_items)
        
        # Compute similarities in parallel
        for i in range(batch_size):
            if not user_item_sets[i]:
                continue
            for j in range(i+1, batch_size):
                if user_item_sets[j]:
                    jaccard = len(user_item_sets[i] & user_item_sets[j]) / len(user_item_sets[i] | user_item_sets[j])
                    adj_matrix[i,j] = adj_matrix[j,i] = jaccard
        
        # Move result to the same device as input
        return adj_matrix.to(device)

    def create_graph_metrics(self, users):
        """Optimized graph metrics creation with MPS support"""
        # Convert users to tensor if it's a list
        if isinstance(users, list):
            users = torch.tensor(users, device=device)
        elif not isinstance(users, torch.Tensor):
            users = torch.tensor([users], device=device)
        
        # Move users to CPU for numpy/scipy operations
        users_cpu = users.cpu()
        metrics = torch.zeros(len(users_cpu), 3)
        
        # Vectorized computation
        for i, user in enumerate(users_cpu):
            interactions = np.array([mat[user.item()].nnz for mat in self.trn_mats])
            metrics[i,0] = sum(interactions) / 100
            metrics[i,1] = (interactions > 0).sum() / len(self.behaviors)
            metrics[i,2] = interactions[-1] / max(interactions[0], 1)
        
        # Move result to the same device as input
        return metrics.to(device)

    def prepare_train_instances(self, max_samples=5000):
        """Optimized training instance preparation"""
        print("Preparing training instances...")
        train_data = []
        
        # Randomly select users
        selected_users = np.random.choice(min(self.n_users, self.max_users), size=min(1000, self.max_users), replace=False)
        
        for user in selected_users:
            pos_items = self.trn_label[user].indices[:2]  # Limit positive items
            
            if len(pos_items) > 0:
                for item in pos_items:
                    behaviors = [float(mat[user, item]) for mat in self.trn_mats]
                    train_data.append([user, item, 1.0] + behaviors)
                    
                    # Limited negative sampling
                    neg_items = np.random.choice(self.n_items, size=2, replace=False)
                    for neg_item in neg_items:
                        behaviors = [float(mat[user, neg_item]) for mat in self.trn_mats]
                        train_data.append([user, neg_item, 0.0] + behaviors)
                        
                    if len(train_data) >= max_samples:
                        break
            
            if len(train_data) >= max_samples:
                break
        
        print(f"Generated {len(train_data)} training instances")
        return np.array(train_data)

    def get_test_instances(self, num_neg_samples=99):
        """Generate test instances with negative sampling"""
        print("Preparing test instances...")
        test_instances = []
        max_test_users = 1000  # Limit number of test users
        
        # Randomly sample test users if there are too many
        test_users = self.tst_users
        if len(test_users) > max_test_users:
            test_users = np.random.choice(test_users, max_test_users, replace=False)
        
        for user in test_users:
            user = int(user)
            pos_item = self.tst_int[user]
            if pos_item is not None:
                pos_item = int(pos_item)
                test_instances.append([user, pos_item, 1.0])
                
                try:
                    # Get negative items
                    all_items = set(range(self.n_items))
                    pos_items = set(int(x) for x in self.trn_label[user].indices)
                    pos_items.add(pos_item)
                    neg_items_pool = list(all_items - pos_items)
                    
                    # Sample negative items
                    n_neg = min(num_neg_samples, len(neg_items_pool))
                    if n_neg > 0:
                        neg_items = np.random.choice(neg_items_pool, size=n_neg, replace=False)
                        for neg_item in neg_items:
                            test_instances.append([user, int(neg_item), 0.0])
                
                except Exception as e:
                    print(f"Error processing test user {user}: {str(e)}")
                    continue
        
        if not test_instances:
            raise ValueError("No test instances were generated!")
        
        test_instances = np.array(test_instances)
        print(f"Generated {len(test_instances)} test instances")
        print(f"Number of unique users: {len(set(test_instances[:,0]))}")
        return test_instances

In [19]:
# this code is optimised only to run in mps based machines , otherwise it will shift the computation to cpus automaticaly
MAX_U = 5000
TOP_N = 10
# --
CHUNK_S=100
TRAIN_SAMP=5000

import os
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, Dropout, LayerNorm
from torch.nn import functional as F
from torch.nn import ModuleList
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from datetime import datetime
from scipy.sparse import csr_matrix

class Scoreformer(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_feedforward, input_dim, num_weights=10, use_weights=True, dropout=0.1):
        super(Scoreformer, self).__init__()
        self.num_weights = num_weights
        self.use_weights = use_weights
        self.input_dim = input_dim
        self.d_model = d_model
        
        # Input projection layers
        self.input_linear = Linear(input_dim, d_model)
        self.dng_projection = Linear(input_dim, d_model)
        
        self.encoder_layer = TransformerEncoderLayer(
            d_model=d_model, 
            nhead=num_heads, 
            dim_feedforward=d_feedforward, 
            dropout=dropout, 
            batch_first=True
        )
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        
        # Final output layers
        self.pre_output = Linear(d_model, d_model)
        self.output_linear = Linear(d_model, 1)
        
        self.dropout = Dropout(dropout)
        self.layer_norm = LayerNorm(d_model)
        
        if self.use_weights:
            self.weight_linears = ModuleList([Linear(input_dim, d_model) for _ in range(num_weights)])

    def compute_neighborhood_similarity(self, adjacency_matrix, x):
        try:
            binary_adj = (adjacency_matrix > 0).float()
            intersection = binary_adj @ binary_adj.T
            row_sums = binary_adj.sum(dim=1, keepdim=True)
            col_sums = binary_adj.sum(dim=0, keepdim=True)
            union = row_sums + col_sums.T - intersection
            similarity = intersection / (union + 1e-8)
            return similarity @ x
        except RuntimeError:
            return torch.zeros_like(x)

    def project_graph_metrics(self, graph_metrics, target_dim):
        if graph_metrics.size(1) < target_dim:
            repeats = (target_dim + graph_metrics.size(1) - 1) // graph_metrics.size(1)
            graph_metrics = graph_metrics.repeat(1, repeats)[:, :target_dim]
        elif graph_metrics.size(1) > target_dim:
            graph_metrics = graph_metrics[:, :target_dim]
        return graph_metrics

    def forward(self, x, adjacency_matrix, graph_metrics, weights=None):
        adjacency_matrix = adjacency_matrix.float()
        graph_metrics = graph_metrics.float()
        batch_size, input_dim = x.shape
        
        if adjacency_matrix.size(0) != batch_size or adjacency_matrix.size(1) != batch_size:
            adjacency_matrix = torch.eye(batch_size, device=x.
            )

        try:
            # Direct connections
            direct_scores = adjacency_matrix @ x
            
            # Neighborhood similarity
            neighborhood_similarity = self.compute_neighborhood_similarity(adjacency_matrix, x)
            
            # Graph structure scores
            if graph_metrics.dim() == 2:
                graph_metrics_projected = self.project_graph_metrics(graph_metrics, input_dim)
                graph_structure_scores = graph_metrics_projected * x
            else:
                graph_structure_scores = torch.zeros_like(x)

            # Combine DNG scores and project to d_model dimension
            dng_scores = direct_scores + neighborhood_similarity + graph_structure_scores
            dng_scores = self.dng_projection(dng_scores)  # Project to d_model dimension
            
            # Process input through transformer
            if self.use_weights and weights is not None:
                weighted_x = torch.zeros_like(x)
                for i, weight in enumerate(weights.T):
                    weighted_x += self.weight_linears[i](x) * weight.unsqueeze(1)
                transformer_input = weighted_x
            else:
                transformer_input = self.input_linear(x)  # Project to d_model dimension

            # Apply transformer
            transformer_input = self.layer_norm(transformer_input)
            transformer_output = self.transformer_encoder(transformer_input.unsqueeze(1)).squeeze(1)
            
            # Combine transformer output with DNG scores
            combined = transformer_output + dng_scores
            combined = self.dropout(combined)
            
            # Final output processing
            output = self.pre_output(combined)
            output = F.relu(output)
            output = self.output_linear(output)
            
            return output.squeeze(-1)  # Return [batch_size] tensor

        except RuntimeError as e:
            print(f"RuntimeError during forward pass: {e}")
            print(f"x shape: {x.shape}, adjacency_matrix shape: {adjacency_matrix.shape}, graph_metrics shape: {graph_metrics.shape}")
            raise

# # cropped dataset
# class MultiBehaviorDataset:
#     def __init__(self, data_path='/Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/beibei01/', max_users=10000):
#         self.data_path = data_path
#         self.behaviors = ['pv', 'cart', 'buy']
#         self.trn_file = data_path + 'trn_'
#         self.tst_file = data_path + 'tst_'
#         self.max_users = max_users  # Limit number of users
        
#         self.load_training_data()
#         self.load_test_data()
    
#     def load_training_data(self):
#         print("Loading training data from:", self.data_path)
#         self.trn_mats = []
#         for beh in self.behaviors:
#             path = self.trn_file + beh
#             print(f"Loading behavior: {beh} from {path}")
#             if not os.path.exists(path):
#                 raise FileNotFoundError(f"File not found: {path}")
#             with open(path, 'rb') as fs:
#                 mat = pickle.load(fs)
#                 if not isinstance(mat, csr_matrix):
#                     mat = csr_matrix(mat)
#                 mat = (mat != 0).astype(np.float32)
#                 self.trn_mats.append(mat)
        
#         self.trn_label = 1 * (self.trn_mats[-1] != 0)
#         self.n_users, self.n_items = self.trn_mats[0].shape
#         self.n_behaviors = len(self.behaviors)
#         print(f"Dataset dimensions: {self.n_users} users, {self.n_items} items")

#     # def create_adjacency_matrix(self, users):
#     #     """Optimized adjacency matrix creation"""
#     #     batch_size = len(users)
#     #     adj_matrix = torch.zeros(batch_size, batch_size, 
# device)
        
#     #     # Pre-compute user item sets
#     #     user_item_sets = []
#     #     for user in users:
#     #         user_items = set()
#     #         for mat in self.trn_mats:
#     #             user_items.update(mat[user].indices)
#     #         user_item_sets.append(user_items)
        
#     #     # Compute similarities in parallel
#     #     for i in range(batch_size):
#     #         if not user_item_sets[i]:
#     #             continue
#     #         for j in range(i+1, batch_size):
#     #             if user_item_sets[j]:
#     #                 jaccard = len(user_item_sets[i] & user_item_sets[j]) / len(user_item_sets[i] | user_item_sets[j])
#     #                 adj_matrix[i,j] = adj_matrix[j,i] = jaccard
        
#     #     return adj_matrix

#     # def create_graph_metrics(self, users):
#     #     """Optimized graph metrics creation"""
#     #     metrics = torch.zeros(len(users), 3, device=device)
        
#     #     # Vectorized computation
#     #     for i, user in enumerate(users):
#     #         interactions = np.array([mat[user].nnz for mat in self.trn_mats])
#     #         metrics[i,0] = sum(interactions) / 100
#     #         metrics[i,1] = (interactions > 0).sum() / len(self.behaviors)
#     #         metrics[i,2] = interactions[-1] / max(interactions[0], 1)
        
#     #     return metrics

#     def create_adjacency_matrix(self, users):
#         """Optimized adjacency matrix creation with MPS support"""
#         # Move users to CPU for numpy/scipy operations
#         users_cpu = users.cpu()
#         batch_size = len(users_cpu)
#         adj_matrix = torch.zeros(batch_size, batch_size)
        
#         # Pre-compute user item sets
#         user_item_sets = []
#         for user in users_cpu:
#             user_items = set()
#             for mat in self.trn_mats:
#                 user_items.update(mat[user.item()].indices)
#             user_item_sets.append(user_items)
        
#         # Compute similarities in parallel
#         for i in range(batch_size):
#             if not user_item_sets[i]:
#                 continue
#             for j in range(i+1, batch_size):
#                 if user_item_sets[j]:
#                     jaccard = len(user_item_sets[i] & user_item_sets[j]) / len(user_item_sets[i] | user_item_sets[j])
#                     adj_matrix[i,j] = adj_matrix[j,i] = jaccard
        
#         # Move result to the same device as input
#         return adj_matrix.to(device)

#     def create_graph_metrics(self, users):
#         """Optimized graph metrics creation with MPS support"""
#         # Move users to CPU for numpy/scipy operations
#         users_cpu = users.cpu()
#         metrics = torch.zeros(len(users_cpu), 3)
        
#         # Vectorized computation
#         for i, user in enumerate(users_cpu):
#             interactions = np.array([mat[user.item()].nnz for mat in self.trn_mats])
#             metrics[i,0] = sum(interactions) / 100
#             metrics[i,1] = (interactions > 0).sum() / len(self.behaviors)
#             metrics[i,2] = interactions[-1] / max(interactions[0], 1)
        
#         # Move result to the same device as input
#         return metrics.to(device)






#         def prepare_train_instances(self, max_samples=5000):
#             """Optimized training instance preparation"""
#             print("Preparing training instances...")
#             train_data = []
            
#             # Randomly select users
#             selected_users = np.random.choice(min(self.n_users, self.max_users), size=min(1000, self.max_users), replace=False)
            
#             for user in selected_users:
#                 pos_items = self.trn_label[user].indices[:2]  # Limit positive items
                
#                 if len(pos_items) > 0:
#                     for item in pos_items:
#                         behaviors = [float(mat[user, item]) for mat in self.trn_mats]
#                         train_data.append([user, item, 1.0] + behaviors)
                        
#                         # Limited negative sampling
#                         neg_items = np.random.choice(self.n_items, size=2, replace=False)
#                         for neg_item in neg_items:
#                             behaviors = [float(mat[user, neg_item]) for mat in self.trn_mats]
#                             train_data.append([user, neg_item, 0.0] + behaviors)
                            
#                         if len(train_data) >= max_samples:
#                             break
                
#                 if len(train_data) >= max_samples:
#                     break
            
#             print(f"Generated {len(train_data)} training instances")
#             return np.array(train_data)
        
#         def load_test_data(self):
#             """Load test data"""
#             test_path = self.tst_file + 'int'
#             print(f"Loading test data from: {test_path}")
#             if not os.path.exists(test_path):
#                 raise FileNotFoundError(f"File not found: {test_path}")
#             with open(test_path, 'rb') as fs:
#                 self.tst_int = np.array(pickle.load(fs))
            
#             self.tst_users = np.reshape(np.argwhere(self.tst_int != None), [-1])
#             # Limit test users for faster processing
#             self.tst_users = self.tst_users[:min(len(self.tst_users), self.max_users)]
#             print(f"Using {len(self.tst_users)} test users")

#         def get_test_instances(self, num_neg_samples=99):
#             """Generate test instances with negative sampling"""
#             print("Preparing test instances...")
#             test_instances = []
            
#             for user in self.tst_users:
#                 pos_item = self.tst_int[user]
#                 if pos_item is not None:
#                     # Add positive instance
#                     test_instances.append([user, pos_item, 1.0])
                    
#                     # Add negative instances
#                     try:
#                         # Get all items and remove positive items
#                         all_items = set(range(self.n_items))
#                         pos_items_train = set(self.trn_label[user].indices)
#                         pos_items_train.add(pos_item)
#                         neg_items_pool = list(all_items - pos_items_train)
                        
#                         # Sample negative items
#                         n_neg = min(num_neg_samples, len(neg_items_pool))
#                         if n_neg > 0:
#                             neg_items = np.random.choice(neg_items_pool, size=n_neg, replace=False)
#                             for neg_item in neg_items:
#                                 test_instances.append([user, neg_item, 0.0])
                    
#                     except Exception as e:
#                         print(f"Error processing test user {user}: {str(e)}")
#                         continue
                    
#                     # Limit the number of test instances for faster processing
#                     if len(test_instances) >= self.max_users * 100:
#                         break
            
#             if not test_instances:
#                 raise ValueError("No test instances were generated!")
                
#             test_instances = np.array(test_instances)
#             print(f"Generated {len(test_instances)} test instances")
#             return test_instances

# def evaluate_model(model, dataset, test_instances, k=10):
#     """Improved evaluation function with better metrics calculation"""
#     model.eval()
#     hits = []
#     ndcgs = []
    
#     # Process test instances in smaller chunks
#     chunk_size = CHUNK_S
#     test_chunks = [test_instances[i:i + chunk_size] for i in range(0, len(test_instances), chunk_size)]
    
#     print(f"Evaluating {len(test_instances)} instances in {len(test_chunks)} chunks...")
    
#     with torch.no_grad():
#         for chunk_idx, chunk in enumerate(test_chunks):
#             if chunk_idx % 50 == 0:  # Reduced frequency of progress updates
#                 print(f"Processing chunk {chunk_idx + 1}/{len(test_chunks)}")
            
#             # Group by user and ensure we have both positive and negative items
#             user_items = {}
#             for inst in chunk:
#                 user = int(inst[0])
#                 item = int(inst[1])
#                 label = float(inst[2])
                
#                 if user not in user_items:
#                     user_items[user] = {'pos': [], 'neg': [], 'all_items': [], 'all_scores': []}
                
#                 user_items[user]['all_items'].append(item)
#                 if label > 0.5:
#                     user_items[user]['pos'].append(item)
#                 else:
#                     user_items[user]['neg'].append(item)
            
#             # Process each user that has both positive and negative items
#             for user, data in user_items.items():
#                 if not data['pos'] or not data['neg']:
#                     continue
                
#                 items = data['all_items']
#                 batch_size = len(items)
                
#                 # Create input features
#                 behaviors = torch.zeros(batch_size, dataset.n_behaviors, device=device)
#                 for i, item in enumerate(items):
#                     for j, mat in enumerate(dataset.trn_mats):
#                         behaviors[i, j] = float(mat[user, item])
                
#                 x = torch.cat([
#                     behaviors,
#                     torch.zeros(batch_size, model.input_dim - behaviors.size(1), device=device)
#                 ], dim=1)
                
#                 # Simplified adjacency matrix for evaluation
#                 adj_matrix = torch.eye(batch_size, device=device)
#                 graph_metrics = dataset.create_graph_metrics([user] * batch_size)
                
#                 try:
#                     predictions = model(x, adj_matrix, graph_metrics)
#                     predictions = predictions.cpu().numpy().flatten()
                    
#                     # Store all scores
#                     for item, score in zip(items, predictions):
#                         data['all_scores'].append((item, score))
                    
#                     # Sort items by score
#                     sorted_items = [x[0] for x in sorted(data['all_scores'], key=lambda x: x[1], reverse=True)]
#                     recommended_items = sorted_items[:k]
                    
#                     # Calculate HR
#                     hit = False
#                     for pos_item in data['pos']:
#                         if pos_item in recommended_items:
#                             hit = True
#                             break
#                     hits.append(hit)
                    
#                     # Calculate NDCG
#                     dcg = 0
#                     idcg = 1  # Ideal DCG for one relevant item
#                     for i, item in enumerate(recommended_items):
#                         if item in data['pos']:
#                             dcg += 1 / np.log2(i + 2)
#                     ndcg = dcg / idcg
#                     ndcgs.append(ndcg)
                    
#                 except Exception as e:
#                     print(f"Error evaluating user {user}: {str(e)}")
#                     continue
    
#     # Calculate final metrics
#     hr = np.mean(hits) if hits else 0
#     ndcg = np.mean(ndcgs) if ndcgs else 0
    
#     # Print detailed statistics
#     print(f"\nEvaluation Statistics:")
#     print(f"Total users evaluated: {len(hits)}")
#     print(f"Number of hits: {sum(hits)}")
#     print(f"Average HR@{k}: {hr:.4f}")
#     print(f"Average NDCG@{k}: {ndcg:.4f}")
    
#     return hr, ndcg


def evaluate_model(model, dataset, test_instances, k=10):
    """Improved evaluation function with better metrics calculation"""
    model.eval()
    hits = []
    ndcgs = []
    
    # Process test instances in smaller chunks
    chunk_size = CHUNK_S
    test_chunks = [test_instances[i:i + chunk_size] for i in range(0, len(test_instances), chunk_size)]
    
    print(f"Evaluating {len(test_instances)} instances in {len(test_chunks)} chunks...")
    
    with torch.no_grad():
        for chunk_idx, chunk in enumerate(test_chunks):
            if chunk_idx % 50 == 0:  # Reduced frequency of progress updates
                print(f"Processing chunk {chunk_idx + 1}/{len(test_chunks)}")
            
            # Group by user and ensure we have both positive and negative items
            user_items = {}
            for inst in chunk:
                user = int(inst[0])
                item = int(inst[1])
                label = float(inst[2])
                
                if user not in user_items:
                    user_items[user] = {'pos': [], 'neg': [], 'all_items': [], 'all_scores': []}
                
                user_items[user]['all_items'].append(item)
                if label > 0.5:
                    user_items[user]['pos'].append(item)
                else:
                    user_items[user]['neg'].append(item)
            
            # Process each user that has both positive and negative items
            for user, data in user_items.items():
                if not data['pos'] or not data['neg']:
                    continue
                
                items = data['all_items']
                batch_size = len(items)
                
                # Create input features
                behaviors = torch.zeros(batch_size, dataset.n_behaviors, device=device)
                for i, item in enumerate(items):
                    for j, mat in enumerate(dataset.trn_mats):
                        behaviors[i, j] = float(mat[user, item])
                
                x = torch.cat([
                    behaviors,
                    torch.zeros(batch_size, model.input_dim - behaviors.size(1), device=device)
                ], dim=1)
                
                # Simplified adjacency matrix for evaluation
                adj_matrix = torch.eye(batch_size, device=device)
                # Convert user to tensor for graph_metrics
                graph_metrics = dataset.create_graph_metrics(torch.tensor([user] * batch_size, device=device))
                
                try:
                    predictions = model(x, adj_matrix, graph_metrics)
                    predictions = predictions.cpu().numpy().flatten()
                    
                    # Store all scores
                    for item, score in zip(items, predictions):
                        data['all_scores'].append((item, score))
                    
                    # Sort items by score
                    sorted_items = [x[0] for x in sorted(data['all_scores'], key=lambda x: x[1], reverse=True)]
                    recommended_items = sorted_items[:k]
                    
                    # Calculate metrics
                    hit = any(item in recommended_items for item in data['pos'])
                    hits.append(hit)
                    
                    # Calculate NDCG
                    dcg = 0
                    idcg = 1  # Ideal DCG for one relevant item
                    for i, item in enumerate(recommended_items):
                        if item in data['pos']:
                            dcg += 1 / np.log2(i + 2)
                    ndcg = dcg / idcg
                    ndcgs.append(ndcg)
                    
                except Exception as e:
                    print(f"Error evaluating user {user}: {str(e)}")
                    continue
    
    # Calculate final metrics
    hr = np.mean(hits) if hits else 0
    ndcg = np.mean(ndcgs) if ndcgs else 0
    
    return hr, ndcg

def get_test_instances(self, num_neg_samples=100):
    """Improved test instance generation"""
    print("Preparing test instances...")
    test_instances = []
    max_test_users = 10000  # Limit number of test users
    
    # Randomly sample test users if there are too many
    test_users = self.tst_users
    if len(test_users) > max_test_users:
        test_users = np.random.choice(test_users, max_test_users, replace=False)
    
    for user in test_users:
        user = int(user)
        pos_item = self.tst_int[user]
        if pos_item is not None:
            pos_item = int(pos_item)
            test_instances.append([user, pos_item, 1.0])
            
            try:
                # Get negative items
                all_items = set(range(self.n_items))
                pos_items = set(int(x) for x in self.trn_label[user].indices)
                pos_items.add(pos_item)
                neg_items_pool = list(all_items - pos_items)
                
                # Sample negative items
                n_neg = min(num_neg_samples, len(neg_items_pool))
                if n_neg > 0:
                    neg_items = np.random.choice(neg_items_pool, size=n_neg, replace=False)
                    for neg_item in neg_items:
                        test_instances.append([user, int(neg_item), 0.0])
            
            except Exception as e:
                print(f"Error processing test user {user}: {str(e)}")
                continue
    
    if not test_instances:
        raise ValueError("No test instances were generated!")
    
    test_instances = np.array(test_instances)
    print(f"Generated {len(test_instances)} test instances")
    print(f"Number of unique users: {len(set(test_instances[:,0]))}")
    return test_instances


def main():
    # Configuration
    config = {
        'd_model': 32,
        'num_heads': 2,
        'num_layers': 1,
        'd_feedforward': 64,
        'input_dim': 64,
        'num_weights': 3,
        'learning_rate': 0.001,
        'weight_decay': 1e-6,
        'dropout': 0.1,
        'gradient_clip': 1.0,
        'max_samples': MAX_U,
        'eval_k': TOP_N
    }
    
    print("Initializing dataset...")
    dataset = MultiBehaviorDataset(max_users=MAX_U)
    train_data = dataset.prepare_train_instances(max_samples=config['max_samples'])
    
    print(f"Using {len(train_data)} training instances")

    model = Scoreformer(
        num_layers=config['num_layers'],
        d_model=config['d_model'],
        num_heads=config['num_heads'],
        d_feedforward=config['d_feedforward'],
        input_dim=config['input_dim'],
        num_weights=config['num_weights'],
        dropout=config['dropout']
    ).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])

    print("Preparing training tensors...")
    users = torch.LongTensor(train_data[:, 0]).to(device)
    items = torch.LongTensor(train_data[:, 1]).to(device)
    labels = torch.FloatTensor(train_data[:, 2]).to(device)
    behaviors = torch.FloatTensor(train_data[:, 3:]).to(device)

    print("Starting training...")
    try:
        model.train()
        start_time = datetime.now()
        
        # Training
        x = torch.cat([
            behaviors,
            torch.zeros(len(users), config['input_dim'] - behaviors.size(1), device=device)
        ], dim=1)
        
        adj_matrix = dataset.create_adjacency_matrix(users)
        graph_metrics = dataset.create_graph_metrics(users)
        
        optimizer.zero_grad()
        predictions = model(x, adj_matrix, graph_metrics)
        predictions = predictions.view(-1)
        labels = labels.view(-1)
        
        loss = criterion(predictions, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config['gradient_clip'])
        optimizer.step()
        
        training_time = (datetime.now() - start_time).total_seconds()
        print(f"Training completed in {training_time:.2f}s")
        print(f"Training loss: {loss.item():.4f}")
        
        # Evaluation
        print("\nStarting evaluation...")
        eval_start_time = datetime.now()
        
        test_instances = dataset.get_test_instances()
        hr, ndcg = evaluate_model(model, dataset, test_instances, k=config['eval_k'])
        
        eval_time = (datetime.now() - eval_start_time).total_seconds()
        print(f"Evaluation completed in {eval_time:.2f}s")
        print(f"HR@{config['eval_k']}: {hr:.4f}")
        print(f"NDCG@{config['eval_k']}: {ndcg:.4f}")
        
        # Save results
        results = {
            'training_time': training_time,
            'eval_time': eval_time,
            'training_loss': loss.item(),
            'hr': hr,
            'ndcg': ndcg
        }
        
        # Save to file
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        with open(f'results_{timestamp}.txt', 'w') as f:
            for key, value in results.items():
                f.write(f"{key}: {value}\n")
        
    except Exception as e:
        print(f"\nError during training: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Initializing dataset...
Loading training data from: /Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/beibei01/
Loading behavior: pv from /Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/beibei01/trn_pv
Loading behavior: cart from /Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/beibei01/trn_cart
Loading behavior: buy from /Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/beibei01/trn_buy
Dataset dimensions: 21716 users, 7977 items
Loading test data from: /Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/beibei01/tst_int
Using 5000 test users
Preparing training instances...


/var/folders/yf/20z1hn994jd04q4kl0gpgh740000gn/T/ipykernel_98133/4112067786.py:21: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  mat = pickle.load(fs)


Generated 5001 training instances
Using 5001 training instances
Preparing training tensors...
Starting training...
Training completed in 101.35s
Training loss: 11.0816

Starting evaluation...
Preparing test instances...
Generated 100000 test instances
Number of unique users: 1000
Evaluating 100000 instances in 1000 chunks...
Processing chunk 1/1000
Processing chunk 51/1000
Processing chunk 101/1000
Processing chunk 151/1000
Processing chunk 201/1000
Processing chunk 251/1000
Processing chunk 301/1000
Processing chunk 351/1000
Processing chunk 401/1000
Processing chunk 451/1000
Processing chunk 501/1000
Processing chunk 551/1000
Processing chunk 601/1000
Processing chunk 651/1000
Processing chunk 701/1000
Processing chunk 751/1000
Processing chunk 801/1000
Processing chunk 851/1000
Processing chunk 901/1000
Processing chunk 951/1000
Evaluation completed in 15.73s
HR@10: 0.9520
NDCG@10: 0.9346


# tmall

In [74]:
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
from datetime import datetime

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class MultiBehaviorDataset:
    def __init__(self, data_path, max_users=5000):
        self.data_path = data_path
        self.behaviors = ['test', 'train']
        self.max_users = max_users
        self.load_data()
    
    def load_data(self):
        print(f"Loading data from {self.data_path}")
        self.trn_mats = []
        for beh in self.behaviors:
            path = os.path.join(self.data_path, f'ijcai2016_koubei_{beh}')
            if not os.path.exists(path):
                raise FileNotFoundError(f"File not found: {path}")
            with open(path, 'rb') as fs:
                mat = pickle.load(fs)
                mat = csr_matrix(mat) if not isinstance(mat, csr_matrix) else mat
                self.trn_mats.append(mat)
        
        self.trn_label = 1 * (self.trn_mats[-1] != 0)
        self.n_users, self.n_items = self.trn_mats[0].shape
        print(f"Loaded dataset with {self.n_users} users and {self.n_items} items")

def evaluate_model(model, test_data, k=10):
    """
    Simplified evaluation function that calculates HR@k and NDCG@k
    """
    model.eval()
    hits, ndcgs = [], []
    
    with torch.no_grad():
        # Group test data by user
        user_items = {}
        for user, item, label in test_data:
            if user not in user_items:
                user_items[user] = {'pos': [], 'neg': []}
            if label == 1:
                user_items[user]['pos'].append(item)
            else:
                user_items[user]['neg'].append(item)
        
        # Calculate metrics for each user
        for user, items in user_items.items():
            if not items['pos'] or not items['neg']:
                continue
                
            # Get all items for this user
            all_items = items['pos'] + items['neg']
            predictions = model.predict(user, all_items)
            
            # Sort items by prediction score
            item_scores = list(zip(all_items, predictions))
            sorted_items = [x[0] for x in sorted(item_scores, key=lambda x: x[1], reverse=True)]
            
            # Calculate HR@k
            recommended_items = sorted_items[:k]
            hit = any(item in recommended_items for item in items['pos'])
            hits.append(hit)
            
            # Calculate NDCG@k
            dcg = sum(1 / np.log2(i + 2) for i, item in enumerate(recommended_items) if item in items['pos'])
            idcg = sum(1 / np.log2(i + 2) for i in range(min(len(items['pos']), k)))
            ndcg = dcg / idcg if idcg > 0 else 0
            ndcgs.append(ndcg)
    
    hr = np.mean(hits)
    ndcg = np.mean(ndcgs)
    return hr, ndcg

def main():
    config = {
        'data_path': '/Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/Tmall/',
        'max_users': 5000,
        'k': 10
    }
    
    try:
        # Load dataset
        dataset = MultiBehaviorDataset(config['data_path'], max_users=config['max_users'])
                
        # Get test data
        test_data = dataset.get_test_instances()
        
        # Evaluate
        start_time = datetime.now()
        hr, ndcg = evaluate_model(model, test_data, k=config['k'])
        eval_time = (datetime.now() - start_time).total_seconds()
        
        print(f"\nEvaluation Results:")
        print(f"HR@{config['k']}: {hr:.4f}")
        print(f"NDCG@{config['k']}: {ndcg:.4f}")
        print(f"Evaluation time: {eval_time:.2f}s")
        
    except FileNotFoundError as e:
        print(f"Error: {str(e)}")
        print("Please ensure the dataset path is correct and the required files exist.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Loading data from /Users/visheshyadav/Documents/GitHub/CoreRec/src/SANDBOX/dataset/Tmall/
An error occurred: could not find MARK
